In [90]:
import numpy as np
import pandas as pd

In [91]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder 

In [92]:
df = pd.read_csv('covid_toy.csv')
df.head()  # age is a numerical col, gender , city is a nominal categorical col and cough, fever is ordinal

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [93]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [94]:
df.isnull().sum() # there are 10 null vals in fever col. so apply simple imputer

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [95]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:5],df['has_covid'],test_size=0.2)

In [96]:
# adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape , x_test_fever.shape

((80, 1), (20, 1))

In [97]:
# Ordinal Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']],dtype=np.int32)
x_train_cough = oe.fit_transform(x_train[['cough']])

# also for the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape,x_test_cough.shape

((80, 1), (20, 1))

In [98]:
#  OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first', sparse=False, dtype=np.int32)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']]) 
#  also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape, x_test_gender_city.shape 

C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


((80, 4), (20, 4))

In [99]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_train.drop(columns=['gender','fever','cough','city']).values


In [100]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)

In [101]:
x_train_transformed

array([[ 25.   ,  99.   ,   0.   ,   0.   ,   1.   ,   0.   ,   1.   ],
       [ 26.   ,  98.   ,   0.   ,   0.   ,   1.   ,   0.   ,   0.   ],
       [ 83.   ,  98.   ,   1.   ,   1.   ,   0.   ,   0.   ,   0.   ],
       [ 49.   ,  99.   ,   0.   ,   0.   ,   0.   ,   0.   ,   1.   ],
       [ 27.   , 100.   ,   1.   ,   1.   ,   0.   ,   0.   ,   0.   ],
       [ 27.   , 100.   ,   1.   ,   0.   ,   1.   ,   0.   ,   0.   ],
       [ 49.   , 101.   ,   0.   ,   1.   ,   0.   ,   0.   ,   0.   ],
       [ 40.   ,  98.   ,   0.   ,   1.   ,   0.   ,   0.   ,   1.   ],
       [ 82.   , 100.625,   1.   ,   0.   ,   1.   ,   0.   ,   0.   ],
       [ 31.   ,  98.   ,   0.   ,   0.   ,   1.   ,   0.   ,   0.   ],
       [ 75.   , 100.625,   0.   ,   1.   ,   0.   ,   0.   ,   0.   ],
       [ 10.   ,  98.   ,   0.   ,   0.   ,   1.   ,   0.   ,   1.   ],
       [ 15.   , 101.   ,   1.   ,   1.   ,   0.   ,   0.   ,   0.   ],
       [  5.   , 100.   ,   0.   ,   0.   ,   1.   ,   0.   ,   

MENTOS ZINDAGI

In [102]:
from sklearn.compose import ColumnTransformer 

In [103]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough') # we can use drop instead of passthrough to drop a column if not needed

In [104]:
transformer.fit_transform(x_train).shape

C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [106]:
transformer.transform(x_test).shape

(20, 7)